In [1]:
import pandas as pd
import string

In [2]:
df = pd.read_csv('data/kc_cleaned_data.csv')
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Import outside data

In [3]:
df_zipcode = pd.read_csv('data/zipcode.csv')

df_zipcode.drop(columns=['Region Type', 'Type'], inplace=True)
df_zipcode = df_zipcode[:82].replace('---', '0.0')

print(df_zipcode.shape)
df_zipcode.head()

(82, 7)


,Region Name,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized
0,King County,"$667,300",0.9%,2.7%,4.5%,8.8%,6.0%
1,98001,"$432,800",1.2%,3.9%,9.4%,9.1%,4.9%
2,98002,"$358,500",1.3%,4.1%,11.5%,10.3%,5.1%
3,98003,"$392,800",1.2%,3.7%,10.1%,9.7%,5.3%
4,98004,"$1,613,200",0.8%,2.5%,4.9%,9.8%,7.1%


In [4]:
df_loc=df.copy()

### Clean price column

In [5]:
def strip_col(dfName, colName):
    replace_col = []
    index = 0
    while index < len(dfName[colName]):
        new = (dfName[colName][index]).replace('$', "").replace(',', "").replace('%',"")
        replace_col.append(new)
        index += 1
    return replace_col

In [6]:
df_zipcode['Current'] = strip_col(df_zipcode, 'Current')
df_zipcode['Month Over Month'] = strip_col(df_zipcode, 'Month Over Month')
df_zipcode['Quarter Over Quarter'] = strip_col(df_zipcode, 'Quarter Over Quarter')
df_zipcode['Year Over Year'] = strip_col(df_zipcode, 'Year Over Year')
df_zipcode['5 Year Annualized'] = strip_col(df_zipcode, '5 Year Annualized')
df_zipcode['10 Year Annualized'] = strip_col(df_zipcode, '10 Year Annualized')

In [7]:
df_zipcode['Current'] = pd.to_numeric(df_zipcode['Current'])
df_zipcode['Month Over Month'] = pd.to_numeric(df_zipcode['Month Over Month'])
df_zipcode['Quarter Over Quarter'] = pd.to_numeric(df_zipcode['Quarter Over Quarter'])
df_zipcode['Year Over Year'] = pd.to_numeric(df_zipcode['Year Over Year'])
df_zipcode['5 Year Annualized'] = pd.to_numeric(df_zipcode['5 Year Annualized'])
df_zipcode['10 Year Annualized'] = pd.to_numeric(df_zipcode['10 Year Annualized'])

In [8]:
df_zipcode.drop(labels=0, axis=0, inplace=True)
df_zipcode['Region Name'] = pd.to_numeric(df_zipcode['Region Name'])


In [9]:
df_zipcode.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81 entries, 1 to 81
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Region Name           81 non-null     int64  
 1   Current               81 non-null     int64  
 2   Month Over Month      81 non-null     float64
 3   Quarter Over Quarter  81 non-null     float64
 4   Year Over Year        81 non-null     float64
 5   5 Year Annualized     81 non-null     float64
 6   10 Year Annualized    81 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 5.1 KB


In [10]:
df_zipcode

,Region Name,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized
1,98001,432800,1.2,3.9,9.4,9.1,4.9
2,98002,358500,1.3,4.1,11.5,10.3,5.1
3,98003,392800,1.2,3.7,10.1,9.7,5.3
4,98004,1613200,0.8,2.5,4.9,9.8,7.1
5,98005,1118500,0.1,0.3,-1.0,8.9,7.1
...,...,...,...,...,...,...,...
77,98178,505100,0.9,2.9,6.0,11.0,6.1
78,98188,428200,1.3,4.3,9.2,11.0,5.5
79,98198,427600,1.2,3.5,8.3,10.0,5.2
80,98199,1013900,0.4,1.7,2.7,7.6,5.9


# Merge

In [11]:
def add_zipdf_to_locdf():
    area_value = []
    area_month = []
    area_quarter = []
    area_year = []
    area_5 = []
    area_10 = []
    index = 0
    while index < len(df_loc['zipcode']):
        loc_val = df_loc['zipcode'][index]
        this_current = (df_zipcode.loc[df_zipcode['Region Name']==loc_val])
        
        area_value.append((pd.Series(this_current['Current']).values)[0])
        area_month.append((pd.Series(this_current['Month Over Month']).values)[0])
        area_quarter.append((pd.Series(this_current['Quarter Over Quarter']).values)[0])
        area_year.append((pd.Series(this_current['Year Over Year']).values)[0])
        area_5.append((pd.Series(this_current['5 Year Annualized']).values)[0])
        area_10.append((pd.Series(this_current['10 Year Annualized']).values)[0])
        
        index+=1
    return area_value, area_month, area_quarter, area_year, area_5, area_10

In [12]:
df_loc['Current'], df_loc['Month Over Month'], df_loc['Quarter Over Quarter'], df_loc['Year Over Year'], df_loc['5 Year Annualized'], df_loc['10 Year Annualized'] = add_zipdf_to_locdf()
df_loc.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,has_basement,is_cust_build,years_since_renovated,year,Current,Month Over Month,Quarter Over Quarter,Year Over Year,5 Year Annualized,10 Year Annualized
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,0,0,0.0,2014,505100,0.9,2.9,6.0,11.0,6.1
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,1,0,29.0,2014,685200,0.7,2.0,1.3,9.0,6.4
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,0,0,0.0,2015,716200,0.8,2.5,3.2,8.5,6.2
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,1,0,0.0,2014,769400,0.7,2.1,3.1,8.1,5.8
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,0,0,0.0,2015,966900,0.9,2.9,4.8,8.0,6.3


In [13]:
df_loc.to_csv('data/kc_data_plus_imported.csv', index=False)

In [14]:
df_loc.drop(columns=['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'sqft_living15', 'sqft_lot15', 'has_basement',
       'is_cust_build', 'years_since_renovated'], inplace=True)

In [15]:
df_loc.to_csv('data/location_price_info.csv', index=False)